In [2]:
import numpy as np
import pandas as pd

In [16]:
hea_22 = pd.read_stata('ch22o_EN_1.0p.dta')
rel_22 = pd.read_stata('cr22o_EN_1.0p.dta')
soc_22 = pd.read_stata('cs22o_EN_1.1p.dta')
fam_22 = pd.read_stata('cf22o_EN_1.0p.dta')
wor_22 = pd.read_stata('cw22o_EN_1.0p.dta')
per_22 = pd.read_stata('cp22n_EN_1.0p.dta')
val_22 = pd.read_stata('cv22n_EN_1.0p.dta')
ass_22 = pd.read_stata('ca22h_EN_1.0p.dta')
inc_22 = pd.read_stata('ci22o_EN_1.0p.dta')
hou_22 = pd.read_stata('cd22o_EN_1.0p.dta')

In [43]:
all_22 = [ass_22, hou_22, fam_22, hea_22, inc_22, per_22, soc_22, val_22, wor_22]

merged_22 = all_22[0]

for all in all_22[1:]:
    merged_22 = pd.merge(merged_22, all, on='nomem_encr', how='inner')

In [44]:
merged_22.head()

,nomem_encr,ca22h_m,ca22h001,ca22h002,ca22h003,ca22h082,ca22h078,ca22h005,ca22h006,ca22h007,...,cw22o496,cw22o497,cw22o498,cw22o499,cw22o500,cw22o501,cw22o502,cw22o503,cw22o504,cw22o505
0,800015.0,202209.0,NaN,NaN,unknown,NaN,yes,no,no,no,...,1 Certainly not,5 Certainly yes,3.0,3.0,1 Certainly not,25-04-2022,23:06:14,25-04-2022,23:27:33,1279.0
1,800127.0,202209.0,NaN,NaN,unknown,NaN,yes,no,no,no,...,2.0,3.0,2.0,4.0,4.0,20-04-2022,13:44:17,20-04-2022,14:31:05,2808.0
2,800170.0,202209.0,NaN,NaN,unknown,NaN,yes,no,no,no,...,1 Certainly not,5 Certainly yes,4.0,5 Certainly yes,5 Certainly yes,12-04-2022,12:31:08,12-04-2022,12:44:52,824.0
3,800186.0,202209.0,NaN,NaN,unknown,NaN,yes,no,no,no,...,4.0,4.0,4.0,4.0,4.0,04-04-2022,13:22:48,04-04-2022,13:27:21,273.0
4,800201.0,202209.0,NaN,NaN,unknown,NaN,yes,no,no,no,...,4.0,2.0,4.0,4.0,2.0,05-04-2022,15:49:27,05-04-2022,20:43:52,17665.0


In [8]:
"""m22_80nan = merged_22.columns[merged_22.isna().mean() > 0.8].tolist()
clean_22 = merged_22.drop(columns=m22_80nan, axis=1)
clean_22.head()"""

,nomem_encr,ca22h_m,ca22h003,ca22h078,ca22h005,ca22h006,ca22h007,ca22h008,ca22h009,ca22h010,...,cw22o496,cw22o497,cw22o498,cw22o499,cw22o500,cw22o501,cw22o502,cw22o503,cw22o504,cw22o505
0,800015.0,202209.0,unknown,yes,no,no,no,no,no,no,...,1 Certainly not,5 Certainly yes,3.0,3.0,1 Certainly not,25-04-2022,23:06:14,25-04-2022,23:27:33,1279.0
1,800127.0,202209.0,unknown,yes,no,no,no,yes,no,no,...,2.0,3.0,2.0,4.0,4.0,20-04-2022,13:44:17,20-04-2022,14:31:05,2808.0
2,800170.0,202209.0,unknown,yes,no,no,no,no,no,no,...,1 Certainly not,5 Certainly yes,4.0,5 Certainly yes,5 Certainly yes,12-04-2022,12:31:08,12-04-2022,12:44:52,824.0
3,800186.0,202209.0,unknown,yes,no,no,no,no,no,no,...,4.0,4.0,4.0,4.0,4.0,04-04-2022,13:22:48,04-04-2022,13:27:21,273.0
4,800201.0,202209.0,unknown,yes,no,no,no,yes,no,no,...,4.0,2.0,4.0,4.0,2.0,05-04-2022,15:49:27,05-04-2022,20:43:52,17665.0


In [48]:
#column combinators (replaces given columns with one containing their avg values)
def average_columns(df, *columns):
    new_column_name = columns[-1]
    
    try:
        selected_columns = list(columns[:-1])
        numeric_columns = df[selected_columns].apply(pd.to_numeric, errors='coerce')
        df[new_column_name] = numeric_columns.mean(axis=1)
    except KeyError as e:
        print(f'KeyError: {e} not found in DataFrame columns.')
        df[new_column_name] = np.nan

    df.drop(columns=list(columns[:-1]), axis=1, inplace=True)

def sum_columns(df, new_column, *columns):
    df[new_column] = df[columns].sum(axis=1)

freq16_map = {'never':1, 'seldom':2, 'sometimes':3, 'often':4, 'mostly':5, 'continuously':6}
freq13_map = {'never':1, 'sometimes':2, 'regularly':3}


Concepts in health:
1. **Subjective health** (ch22o004 to sub_hea)
2. **Relative health** (ch22o005 to rel_hea)
3. Life expectancy
4. **Mental health** (all neg as men_hea_neg and and all pos as men_hea_pos)
5. Height, weight
6. **Chronical problems** (ch22o018 to disability)
7. Health problems
8. Impediment in labour
9. **Sex drugs alcohol** (ch22o126 to smoker, ch22o133 to alcohol, all substances to subs)
10. Medication
11. Physical activity - only until 2018
12. Health improving behaviour - - only until 2018
13. Use of healthcare
14. Sight and hearing
14. Health insurance

In [40]:
hea_22.head()

,nomem_encr,ch22o_m,ch22o001,ch22o002,ch22o003,ch22o004,ch22o005,ch22o006,ch22o007,ch22o008,...,ch22o250,ch22o251,ch22o252,ch22o253,ch22o254,ch22o255,ch22o256,ch22o257,ch22o258,ch22o259
0,800001.0,202211.0,male,61.0,has paid job,moderate,the same,5.0,3.0,NaN,...,2.0,4.0,2.0,4.0,4.0,22-11-2022,17:07:13,22-11-2022,17:31:06,1433.0
1,800002.0,202211.0,male,45.0,has paid job,very good,the same,9.0,8.0,NaN,...,2.0,4.0,2.0,3.0,3.0,11-11-2022,18:26:22,11-11-2022,18:42:55,993.0
2,800009.0,202211.0,male,67.0,has paid job,moderate,somewhat poorer,NaN,7.0,7.0,...,certainly not,certainly yes,certainly not,3.0,3.0,28-11-2022,21:09:49,28-11-2022,21:17:26,457.0
3,800015.0,202211.0,male,60.0,has paid job,good,the same,8.0,7.0,NaN,...,certainly not,certainly yes,certainly yes,certainly yes,certainly not,13-11-2022,14:29:52,13-11-2022,14:35:51,359.0
4,800028.0,202211.0,male,72.0,has no paid job,good,the same,NaN,NaN,7.0,...,certainly not,certainly yes,3.0,3.0,3.0,07-11-2022,13:46:06,07-11-2022,13:52:03,357.0


In [45]:
print(merged_22[['ch22o011','ch22o014','ch22o012','ch22o013', 'ch22o015', 'ch22o159', 'ch22o160', 'ch22o161', 'ch22o161', 'ch22o162', 'ch22o163', 'ch22o270']].head())

    ch22o011   ch22o014   ch22o012   ch22o013 ch22o015 ch22o159 ch22o160  \
0      never      never      never      often    often    never    never   
1  sometimes     mostly  sometimes     seldom   seldom    never    never   
2      never     seldom     seldom     mostly   mostly    never    never   
3      often  sometimes  sometimes  sometimes    often    never    never   
4     seldom     seldom     seldom      often   seldom    never    never   

  ch22o161 ch22o161 ch22o162 ch22o163 ch22o270  
0    never    never    never    never    never  
1    never    never    never    never    never  
2    never    never    never    never    never  
3    never    never    never    never    never  
4    never    never    never    never    never  


In [31]:
hea_toavg = [('ch22o011','ch22o014','ch22o012', 'men_hea_neg'),('ch22o013', 'ch22o015', 'men_hea_pos'), ('ch22o159', 'ch22o160', 'ch22o161', 'ch22o161', 'ch22o162', 'ch22o163', 'ch22o270', 'subs')]

for column_set in hea_toavg:
    average_columns(merged_22, *column_set)

print(merged_22.head())

   nomem_encr   ca22h_m ca22h001 ca22h002 ca22h003  ca22h082 ca22h078  \
0    800015.0  202209.0      NaN      NaN  unknown       NaN      yes   
1    800127.0  202209.0      NaN      NaN  unknown       NaN      yes   
2    800170.0  202209.0      NaN      NaN  unknown       NaN      yes   
3    800186.0  202209.0      NaN      NaN  unknown       NaN      yes   
4    800201.0  202209.0      NaN      NaN  unknown       NaN      yes   

  ca22h005 ca22h006 ca22h007  ...         cw22o499         cw22o500  \
0       no       no       no  ...              3.0  1 Certainly not   
1       no       no       no  ...              4.0              4.0   
2       no       no       no  ...  5 Certainly yes  5 Certainly yes   
3       no       no       no  ...              4.0              4.0   
4       no       no       no  ...              4.0              2.0   

     cw22o501  cw22o502    cw22o503  cw22o504 cw22o505 men_hea_neg  \
0  25-04-2022  23:06:14  25-04-2022  23:27:33   1279.0         N

In [ ]:
hea_con = {'ch22o004': 'sub_hea', 'ch22o005': 'rel_hea', 'ch22o018': 'disability', 'ch22o020':'mob_issues', 'ch22o126':'smoker', 'ch22o133':'alcohol',  }


In [ ]:
#no of cars, motorcycles, boats, (static) caravans is ca22h008
#value is ca22h023 ??????????????what now